In [464]:
import pandas as pd
import ast
import numpy as np
import datetime as dt
from bs4 import BeautifulSoup
import requests
from textblob import TextBlob
import emoji

# Limpieza y transformación del dataset de output steam games

In [465]:
# pasamos los archivos a un Dataframe
dfoutput=pd.read_json(r'output_steam_games.json',lines=True, encoding='MacRoman')

In [466]:
dfoutput.iloc[-1]

publisher                                                    None
genres                                                       None
app_name                                              Maze Run VR
title                                                        None
url             http://store.steampowered.com/app/681550/Maze_...
release_date                                                 None
tags            [Early Access, Adventure, Indie, Action, Simul...
reviews_url     http://steamcommunity.com/app/681550/reviews/?...
specs           [Single-player, Stats, Steam Leaderboards, HTC...
price                                                        4.99
early_access                                                  1.0
id                                                       681550.0
developer                                                    None
Name: 120444, dtype: object

In [467]:
#conservamos las columnas que nos van a servir para nuestros objetivos
dfjuegos=pd.DataFrame(dfoutput[['id','app_name','developer','publisher','release_date']].drop_duplicates()[1:])

In [468]:
dfjuegos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32133 entries, 88310 to 120444
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            32132 non-null  float64
 1   app_name      32132 non-null  object 
 2   developer     28835 non-null  object 
 3   publisher     24082 non-null  object 
 4   release_date  30067 non-null  object 
dtypes: float64(1), object(4)
memory usage: 1.5+ MB


In [469]:
#buscamos nulos en el dataframe para tratarlos
dfjuegos.isnull().sum()

id                 1
app_name           1
developer       3298
publisher       8051
release_date    2066
dtype: int64

In [470]:
#vemos un nulo en id
dfjuegos[dfjuegos.id.isnull()]

,id,app_name,developer,publisher,release_date
119271,NaN,Batman: Arkham City - Game of the Year Edition,"Rocksteady Studios,Feral Interactive (Mac)","Warner Bros. Interactive Entertainment, Feral ...",2012-09-07


In [471]:
#verficamos si esta duplicado y efectivamente parece que lo esta, comprobamos quitando la columna id
print(dfjuegos[dfjuegos.app_name==dfjuegos.app_name[dfjuegos.id.isnull()].iloc[0]])
dfjuegos.drop(index=dfjuegos.loc[dfjuegos.id.isnull()].index[0],inplace=True)
dfjuegos.isnull().sum()

              id                                        app_name  \
89378   200260.0  Batman: Arkham City - Game of the Year Edition   
119271       NaN  Batman: Arkham City - Game of the Year Edition   

                                         developer  \
89378   Rocksteady Studios,Feral Interactive (Mac)   
119271  Rocksteady Studios,Feral Interactive (Mac)   

                                                publisher release_date  
89378   Warner Bros. Interactive Entertainment, Feral ...   2012-09-07  
119271  Warner Bros. Interactive Entertainment, Feral ...   2012-09-07  


id                 0
app_name           1
developer       3298
publisher       8051
release_date    2066
dtype: int64

In [472]:
#cambiamos el tipo de dato de la columna id a int
dfjuegos['id']=dfjuegos.id.astype(int)
dfjuegos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32132 entries, 88310 to 120444
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            32132 non-null  int32 
 1   app_name      32131 non-null  object
 2   developer     28834 non-null  object
 3   publisher     24081 non-null  object
 4   release_date  30066 non-null  object
dtypes: int32(1), object(4)
memory usage: 1.3+ MB


In [473]:
#vemos 1 nulo en app_name
dfjuegos[dfjuegos.app_name.isnull()]

,id,app_name,developer,publisher,release_date
90890,317160,None,None,None,2014-08-26


In [474]:
# el nulo en app_name lo sacamos del url del juego
url=dfoutput.url[dfoutput.id==dfjuegos.id[dfjuegos.app_name.isnull()].iloc[0]].iloc[0]
url

'http://store.steampowered.com/app/317160/_/'

In [475]:
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')
titulo=soup.head.title
titulo=titulo.text
titulo=titulo.removesuffix(' on Steam')
titulo

'Duet'

In [476]:
#colocamos el nombre de la aplicacion faltante obtenido de la pagina web y verificamos que la columna app_name ya no tiene nulos
dfjuegos.loc[dfjuegos.app_name.isnull(),'app_name']=titulo
dfjuegos.isnull().sum()

id                 0
app_name           0
developer       3298
publisher       8051
release_date    2066
dtype: int64

In [477]:
# sustituimos valores nulos de la columna desarrolladores con la columna de publihser, solo en las filas donde no hya nulos en publisher
dfjuegos.loc[(dfjuegos.developer.isnull())&(~dfjuegos.publisher.isnull()),'developer']=dfjuegos.publisher
#sustituimos lo valores nulos restantes con 'N/D'
dfjuegos.loc[(dfjuegos.developer.isnull()),'developer']='N/D'
dfjuegos.isnull().sum()

id                 0
app_name           0
developer          0
publisher       8051
release_date    2066
dtype: int64

In [478]:
#quitamos la columna publisher, ya que solo nos interesa la de desarrollador
dfjuegos.drop(columns='publisher',inplace=True)
dfjuegos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32132 entries, 88310 to 120444
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            32132 non-null  int32 
 1   app_name      32132 non-null  object
 2   developer     32132 non-null  object
 3   release_date  30066 non-null  object
dtypes: int32(1), object(3)
memory usage: 2.1+ MB


In [479]:
#sacamos el año de lanzamiento de la columna release_date y quitamos la columna release_date
dfjuegos['Año_lanzamiento']=dfjuegos.release_date.str.split('-',expand=True)[0]


In [480]:
dfjuegos.head()

,id,app_name,developer,release_date,Año_lanzamiento
88310,761140,Lost Summoner Kitty,Kotoshiro,2018-01-04,2018
88311,643980,Ironbound,Secret Level SRL,2018-01-04,2018
88312,670290,Real Pool 3D - Poolians,Poolians.com,2017-07-24,2017
88313,767400,弹炸人2222,彼岸领域,2017-12-07,2017
88314,773570,Log Challenge,N/D,None,None


In [481]:
#renombramos los nombres de las columnas
dfjuegos.rename(columns={'Año lanzamiento':'Año_lanzamiento','app_name':'Nombre','developer':'Desarrollador','id':'Id_juego'},inplace=True)

In [440]:
#hacemos una lista de los años que hay en la columna Año_de_lanzamiento
años=[]
for a in dfjuegos.Año_lanzamiento.sort_values().unique():
    try:
        int(a)
    except:
        continue
    else:
        años.append(a)
años[0:5]

['1970', '1975', '1980', '1981', '1982']

In [483]:
#cambiamos las filas que tienen el año en una parte de su cadena a contener solo el dato del año
for a in años:
    dfjuegos.loc[dfjuegos.Año_lanzamiento.str.contains(a,na=False),'Año_lanzamiento']=a

In [484]:
dfjuegos.Año_lanzamiento.sort_values().unique()

array(['"""Soon"""',
       '0̵1̴0̵0̶1̷0̶0̵0̴ ̴0̶0̶1̶1̶0̷0̶1̵1̴ ̸0̶0̶1̶1̵0̶1̷0̴0̵ ̴0̶1̷0̸1̵0̷0̴1̶0̴ ̴0̷0̴1̷1̶0̶1̵1̷1̵ ̵',
       '14 July', '1970', '1975', '1980', '1981', '1982', '1983', '1984',
       '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992',
       '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', '2019', '2021', 'Beta测试已开启',
       "C'est bientôt...                    (ou pas)", 'Coming Soon',
       'Coming Soon!', 'Coming Soon/Próximamente', 'Coming soon',
       'Coming soon..', 'Comming Soon',
       'Datachunks conflicted. Be vigiliant.', 'Demo coming soon.',
       'Demo is available now!', 'Early Access Starting Soon!',
       'Early Access soon', 'Not yet available', 'Play Beta in demo!',
       'Please wait warmly', 'Release Date TBA', 'SOON', 'SOON™', 'Soon',
       'Soon..'

In [311]:
# se hace una lista de los datos que asiguen pendientes en la columna año de lanzamiento, y haciendo investigacion se encuentra que los juegos fueron lanzados del 2018 en adelante
datossinaño=[]
for a in dfjuegos.Año_lanzamiento.sort_values().unique():
    if a not in años:
        datossinaño.append(a)
datossinaño

['"""Soon"""',
 '0̵1̴0̵0̶1̷0̶0̵0̴ ̴0̶0̶1̶1̶0̷0̶1̵1̴ ̸0̶0̶1̶1̵0̶1̷0̴0̵ ̴0̶1̷0̸1̵0̷0̴1̶0̴ ̴0̷0̴1̷1̶0̶1̵1̷1̵ ̵',
 '14 July',
 'Beta测试已开启',
 "C'est bientôt...                    (ou pas)",
 'Coming Soon',
 'Coming Soon!',
 'Coming Soon/Próximamente',
 'Coming soon',
 'Coming soon..',
 'Comming Soon',
 'Datachunks conflicted. Be vigiliant.',
 'Demo coming soon.',
 'Demo is available now!',
 'Early Access Starting Soon!',
 'Early Access soon',
 'Not yet available',
 'Play Beta in demo!',
 'Please wait warmly',
 'Release Date TBA',
 'SOON',
 'SOON™',
 'Soon',
 'Soon..',
 'Soon™',
 'TBA',
 'TBD',
 'To Be Announced',
 'To be Announced',
 'To be announced',
 'When it is finished',
 'When it is ready',
 "When it's done",
 "When it's done!",
 'coming soon',
 'early access',
 'soon',
 '预热群52756441',
 None]

In [492]:
dfjuegos[dfjuegos.Año_lanzamiento=='14 July']

,Id_juego,Nombre,Desarrollador,release_date,Año_lanzamiento
98793,658340,Edge of Insanity,Grotta Game Studio,14 July,14 July


In [312]:
#falta comprobar si los videjugos que no tienen año son relevantes para las funciones y en caso que no, hare la conversion despues al año 2018 que es al año en que la mayoria se lanzaron

In [313]:
# unimos los generos al dataframe de juegos
dfjuegos=dfjuegos.join(dfoutput[['genres','tags']],how='inner')


In [314]:
dfjuegos.head()

,Id_juego,Nombre,Desarrollador,Año_lanzamiento,genres,tags
88310,761140,Lost Summoner Kitty,Kotoshiro,2018,"[Action, Casual, Indie, Simulation, Strategy]","[Strategy, Action, Indie, Casual, Simulation]"
88311,643980,Ironbound,Secret Level SRL,2018,"[Free to Play, Indie, RPG, Strategy]","[Free to Play, Strategy, Indie, RPG, Card Game..."
88312,670290,Real Pool 3D - Poolians,Poolians.com,2017,"[Casual, Free to Play, Indie, Simulation, Sports]","[Free to Play, Simulation, Sports, Casual, Ind..."
88313,767400,弹炸人2222,彼岸领域,2017,"[Action, Adventure, Casual]","[Action, Adventure, Casual]"
88314,773570,Log Challenge,N/D,None,None,"[Action, Indie, Casual, Sports]"


In [315]:
# creamos una lista con los generos existentes
generos=[]
for a in dfjuegos.genres.dropna().drop_duplicates():
    for b in a:
        if b not in generos:
            generos.append(b)
generos=sorted(generos)
print(generos)

['Accounting', 'Action', 'Adventure', 'Animation &amp; Modeling', 'Audio Production', 'Casual', 'Design &amp; Illustration', 'Early Access', 'Education', 'Free to Play', 'Indie', 'Massively Multiplayer', 'Photo Editing', 'RPG', 'Racing', 'Simulation', 'Software Training', 'Sports', 'Strategy', 'Utilities', 'Video Production', 'Web Publishing']


In [316]:
# En la columna tags tenemos algunos valores de los generos que faltan en la columna generos
#los tomareos de ahi haciendonuna lista solo con los generos que tenenos ya en la columna generos
dfjuegos.loc[(dfjuegos.genres.isnull())&(~dfjuegos.tags.isnull()),['tags']].apply(lambda x: [a for a in x if a in generos])
dfjuegos.loc[(dfjuegos.genres.isnull())&(~dfjuegos.tags.isnull()),['tags']]
            

,tags
88314,"[Action, Indie, Casual, Sports]"
88321,[Casual]
88329,"[Early Access, Indie, VR]"
88330,"[Early Access, Action, Adventure, Indie, Casual]"
88332,"[Early Access, Strategy, Action, Indie, Casual..."
...,...
120428,"[Design & Illustration, Tutorial]"
120429,"[Design & Illustration, Tutorial]"
120430,[Movie]
120431,"[Design & Illustration, Tutorial]"


In [317]:
# una vez que tenemos en tags solo los generos que falatan de la columna genres, los sustituimos

dfjuegos.loc[(dfjuegos.genres.isnull())&(~dfjuegos.tags.isnull()),['genres']]=dfjuegos.tags

dfjuegos.isnull().sum()

Id_juego              0
Nombre                0
Desarrollador         0
Año_lanzamiento    2066
genres              138
tags                162
dtype: int64

In [318]:
# el resto de valores nulos de la columna genres los sustituimos por el valor 'N/D' y eliminamos la columna tagss, ya que ya no la utlizaremos
dfjuegos.loc[(dfjuegos.genres.isnull()),'genres']='N/D'
dfjuegos.drop(columns='tags',inplace=True)
dfjuegos.isnull().sum()


Id_juego              0
Nombre                0
Desarrollador         0
Año_lanzamiento    2066
genres                0
dtype: int64

In [319]:
#cambiamos el nombre de la columna genres a Género
dfjuegos.rename(columns={'genres':'Género'},inplace=True)
dfjuegos.columns

Index(['Id_juego', 'Nombre', 'Desarrollador', 'Año_lanzamiento', 'Género'], dtype='object')

In [673]:
dfjuegos.to_csv('Juegos_steam.csv')

# ETl de dataset users items

In [264]:
#abirmos el rchivo de items en un dataframe
with open('australian_users_items.json',encoding='MacRoman') as archivo_json:
    contenido_json = archivo_json.readlines()
lisitems=[eval(line.strip())for line in contenido_json]
dfitems=pd.DataFrame(lisitems)


In [270]:
# Se desanida la columna de items y se pasa a un Dataframe el cual contiene la informacion de cada juego jugado por el usuario, como horas jugadas y el id del juego
lista_items=[] 
lista_items2=[]
for a in dfitems['items']:
    lista_items.append(ast.literal_eval(str(a)))
for a in range(len(lista_items)):
    for b in range(len(lista_items[a])):
        lista_items[a][b]['user_id']=lisitems[a]['user_id']
        lista_items2.append(lista_items[a][b])
        

In [273]:
dfitems=pd.DataFrame(lista_items2)
dfitems.head()

,item_id,item_name,playtime_forever,playtime_2weeks,user_id
0,10,Counter-Strike,6,0,76561197970982479
1,20,Team Fortress Classic,0,0,76561197970982479
2,30,Day of Defeat,7,0,76561197970982479
3,40,Deathmatch Classic,0,0,76561197970982479
4,50,Half-Life: Opposing Force,0,0,76561197970982479


In [274]:
#observamos que hay filas duplicadas en los items
dfitems.duplicated().sum()

59104

In [275]:
#se procede a eliminar duplicados del dataframe de items y verificamos que se eliminaron
dfitems.drop_duplicates(inplace=True,ignore_index=True)
dfitems.duplicated().sum()

0

In [286]:
#cambiamos el tipo de dato a int de la columna de ids del item
dfitems.item_id=dfitems.item_id.astype(int)

In [287]:
dfitems.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094105 entries, 0 to 5094104
Data columns (total 5 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   item_id           int32 
 1   item_name         object
 2   playtime_forever  int64 
 3   playtime_2weeks   int64 
 4   user_id           object
dtypes: int32(1), int64(2), object(2)
memory usage: 174.9+ MB


In [263]:
dfitems.isnull().sum()

item_id             0
item_name           0
playtime_forever    0
playtime_2weeks     0
user_id             0
dtype: int64

In [321]:
#cambiamos nombres de las columnas 
columnas_items={'item_id':'Id_juego','item_name':'Nombre_juego','playtime_forever':'Horas_jugadas','user_id':'Id_usuario'}
dfitems.rename(columns=columnas_items,inplace=True)
dfitems.columns

Index(['Id_juego', 'Nombre_juego', 'Horas_jugadas', 'playtime_2weeks',
       'Id_usuario'],
      dtype='object')

In [262]:
dfitems=pd.read_csv('items_por_usuario.csv')
dfitems.head()

,item_id,item_name,playtime_forever,playtime_2weeks,user_id
0,10,Counter-Strike,6,0,76561197970982479
1,20,Team Fortress Classic,0,0,76561197970982479
2,30,Day of Defeat,7,0,76561197970982479
3,40,Deathmatch Classic,0,0,76561197970982479
4,50,Half-Life: Opposing Force,0,0,76561197970982479


In [674]:
# Se guarda en un csv el dataframe con los items desanidados y el id de usuario
dfitems.to_csv('items_por_usuario.csv',index=False)

# ETL reviews

In [640]:
# abrimos el archivo reviews
with open('australian_user_reviews.json', encoding='utf-8') as archivo_json:
    contenido_json = archivo_json.readlines()
lisreviews=list(map(ast.literal_eval,contenido_json))
dfreviews=pd.DataFrame(lisreviews)
dfreviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [641]:
# Se desanida la columna de items y se pasa a un Dataframe el cual contiene la informacion de cada juego jugado por el usuario, como horas jugadas y el id del juego
lista_reviews=[] 
lista_reviews2=[]
for a in dfreviews['reviews']:
    lista_reviews.append(ast.literal_eval(str(a)))
for a in range(len(lista_reviews)):
    for b in range(len(lista_reviews[a])):
        lista_reviews[a][b]['user_id']=lisreviews[a]['user_id']
        lista_reviews2.append(lista_reviews[a][b])
dfreviews=pd.DataFrame(lista_reviews2)
dfreviews.head()

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637


In [642]:
dfreviews.funny.unique()

array(['', '3 people found this review funny',
       '1 person found this review funny',
       '2 people found this review funny',
       '5 people found this review funny',
       '17 people found this review funny',
       '524 people found this review funny',
       '4 people found this review funny',
       '12 people found this review funny',
       '6 people found this review funny',
       '7 people found this review funny',
       '103 people found this review funny',
       '16 people found this review funny',
       '34 people found this review funny',
       '9 people found this review funny',
       '45 people found this review funny',
       '11 people found this review funny',
       '97 people found this review funny',
       '207 people found this review funny',
       '718 people found this review funny',
       '23 people found this review funny',
       '100 people found this review funny',
       '8 people found this review funny',
       '19 people found this rev

In [643]:
# quitamos las columnas que no vamos a utiizar para las funciones requeridas
dfreviews.drop(columns=['funny','last_edited','helpful'],inplace=True)
dfreviews.head()

,posted,item_id,recommend,review,user_id
0,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...,76561197970982479
1,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.,76561197970982479
2,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
3,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...,js41637
4,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...,js41637


In [644]:
#corroboramos si hay filas duplicadas y eliminamos
print('filas duplicadas: ',dfreviews.duplicated().sum())
dfreviews.drop_duplicates(inplace=True,ignore_index=True)
print('filas duplicadas despues de hacer drop: ',dfreviews.duplicated().sum())

filas duplicadas:  874
filas duplicadas despues de hacer drop:  0


In [645]:
#buscamos valores nulos del dataframe y corroboramos que no existen
dfreviews.isnull().sum()

posted       0
item_id      0
recommend    0
review       0
user_id      0
dtype: int64

In [646]:
#quitamos la palabra posted, las comas y puntos
dfreviews.posted=dfreviews.posted.str.removeprefix('Posted ').str.replace(',','').str.replace('.','')

In [647]:
#sacamos el año de la columna posted
dfreviews['Año_review']=dfreviews.posted.str.split(expand=True)[2]
dfreviews.head()

,posted,item_id,recommend,review,user_id,Año_review
0,November 5 2011,1250,True,Simple yet with great replayability. In my opi...,76561197970982479,2011
1,July 15 2011,22200,True,It's unique and worth a playthrough.,76561197970982479,2011
2,April 21 2011,43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,2011
3,June 24 2014,251610,True,I know what you think when you see this title ...,js41637,2014
4,September 8 2013,227300,True,For a simple (it's actually not all that simpl...,js41637,2013


In [648]:
# En la platfroma Steam, los reviews que no tienen año indican que fueron realizados en el año e curso
#lo que nos indica que el año mas grande en la lista de años unicos es el 2015, el año en curso es el 2016
print(f'valores nulos en la columna Año_review: {dfreviews.Año_review.isnull().sum()}')
print(f'ulitmo año especificado en los datos: {dfreviews.Año_review.sort_values(ascending=False).unique()[0]}')
#procedemos a relllenar los años faltantes con 2016
dfreviews.Año_review.fillna('2016',inplace=True)
#verificamos que ya se cambio el dato y que no hay nulos
print(f'valores nulos en la columna Año_review despues de cambiar nulos: {dfreviews.Año_review.isnull().sum()}')
print(f'ulitmo año especificado en los datos: {dfreviews.Año_review.sort_values(ascending=False).unique()[0]}')

valores nulos en la columna Año_review: 9933
ulitmo año especificado en los datos: 2015
valores nulos en la columna Año_review despues de cambiar nulos: 0
ulitmo año especificado en los datos: 2016


In [658]:
#checamos que las reviews contienen emojis, lo cual afectara al analisis de sentimientos
print(f'reviews con emojis: {dfreviews.review.apply(lambda x: emoji.emoji_count(x)).sum()}')
dfreviews['review_sin_emojis']=dfreviews.review.apply(lambda x: emoji.demojize(x))
print(f'se verifian que la nueva columna no tenga emojis: reviews sin emojis={dfreviews.review_sin_emojis.apply(lambda x: emoji.emoji_count(x)).sum()}')

reviews con emojis: 37235
se verifian que la nueva columna no tenga emojis: reviews sin emojis=0


In [664]:
def get_sentiment_label(text):
    analysis = TextBlob(text)
    polarity = analysis.sentiment.polarity
    
    if polarity < -0.2:
        return 0  # Malo
    elif polarity > 0.2:
        return 2  # Positivo
    else:
        return 1  # Neutral

# Aplicar la función a la columna 'review' y crear la nueva columna 'sentiment_analysis'
dfreviews['sentiment_analysis'] = dfreviews['review'].apply(get_sentiment_label)

# Mostrar las primeras filas para verificar los cambios
dfreviews.head()

,posted,item_id,recommend,review,user_id,Año_review,review_sin_emojis,sentiment_analysis
0,November 5 2011,1250,True,Simple yet with great replayability. In my opi...,76561197970982479,2011,Simple yet with great replayability. In my opi...,1
1,July 15 2011,22200,True,It's unique and worth a playthrough.,76561197970982479,2011,It's unique and worth a playthrough.,2
2,April 21 2011,43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,2011,Great atmosphere. The gunplay can be a bit chu...,1
3,June 24 2014,251610,True,I know what you think when you see this title ...,js41637,2014,I know what you think when you see this title ...,1
4,September 8 2013,227300,True,For a simple (it's actually not all that simpl...,js41637,2013,For a simple (it's actually not all that simpl...,1


In [651]:
# Función para obtener una lista de emojis únicos de un texto
def obtener_emojis_unicos(texto):
    return list(set(c for c in texto if c in emoji.EMOJI_DATA))

# Aplicar la función a la columna 'comentarios' del DataFrame
emojis_unicos = dfreviews.review.apply(obtener_emojis_unicos)

# Convertir la lista de listas resultante en una lista plana de emojis únicos
emojis_unicos = list(set([emoji for sublist in emojis_unicos for emoji in sublist]))

# Imprimir la lista de emojis únicos
print(emojis_unicos)


['™', '⌛', '👍', '🐐', '♥', '♀', '☹', '☑', '👌', '🏻', '☺', '❤', '😄', '👈', '✔', '💋', '®', '😊', '😀', '😋', '💀', '😃', '©', '◀', '👀', '♠', '🐥', '💯']


In [657]:
#verificamos que las reviews donde no se escribió nada tengan 1 como valor como esta indicado en la consigna
dfreviews.sentiment_analysis[(dfreviews.review=='')|(dfreviews.review==' ')].value_counts()

sentiment_analysis
1    38
Name: count, dtype: int64

In [665]:
print('analisis de sentimientos donde la recomendacion es True y el sentimiento es positivo= ',dfreviews['sentiment_analysis'][(dfreviews.recommend==True) & (dfreviews.sentiment_analysis==2)].count())
print('analisis de sentimientos donde la recomendacion es False y el sentimiento es negativo= ',dfreviews['sentiment_analysis'][(dfreviews.recommend==False) & (dfreviews.sentiment_analysis==0)].count())
print('analisis de sentimientos donde la recomendacion es True y el sentimiento es neutral= ',dfreviews['sentiment_analysis'][(dfreviews.recommend==True) & (dfreviews.sentiment_analysis==1)].count())
print('analisis de sentimientos donde la recomendacion es False y el sentimiento es neutral= ',dfreviews['sentiment_analysis'][(dfreviews.recommend==False) & (dfreviews.sentiment_analysis==1)].count())
print('analisis de sentimientos donde la recomendacion es False y el sentimiento es positivo= ', dfreviews['sentiment_analysis'][(dfreviews.recommend==False) & (dfreviews.sentiment_analysis==2)].count())
print('analisis de sentimientos donde la recomendacion es True y el sentimiento es negativo= ',dfreviews['sentiment_analysis'][(dfreviews.recommend==True) & (dfreviews.sentiment_analysis==0)].count())
print(sum([(dfreviews['sentiment_analysis'][(dfreviews.recommend==True) & (dfreviews.sentiment_analysis==2)].count()),
           dfreviews['sentiment_analysis'][(dfreviews.recommend==False) & (dfreviews.sentiment_analysis==0)].count(),
           dfreviews['sentiment_analysis'][(dfreviews.recommend==True) & (dfreviews.sentiment_analysis==1)].count(),
           dfreviews['sentiment_analysis'][(dfreviews.recommend==False) & (dfreviews.sentiment_analysis==1)].count(),
           dfreviews['sentiment_analysis'][(dfreviews.recommend==False) & (dfreviews.sentiment_analysis==2)].count(),
           dfreviews['sentiment_analysis'][(dfreviews.recommend==True) & (dfreviews.sentiment_analysis==0)].count(),
           ]))


analisis de sentimientos donde la recomendacion es True y el sentimiento es positivo=  16812
analisis de sentimientos donde la recomendacion es False y el sentimiento es negativo=  1688
analisis de sentimientos donde la recomendacion es True y el sentimiento es neutral=  31434
analisis de sentimientos donde la recomendacion es False y el sentimiento es neutral=  4444
analisis de sentimientos donde la recomendacion es False y el sentimiento es positivo=  607
analisis de sentimientos donde la recomendacion es True y el sentimiento es negativo=  3446
58431


In [670]:
# cambiamos el tipo de dato de la columna item_id y de Año_review
dfreviews.item_id=dfreviews.item_id.astype(int)
dfreviews.Año_review= pd.to_datetime(dfreviews.Año_review, format='%Y')
dfreviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58431 entries, 0 to 58430
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   posted              58431 non-null  object        
 1   item_id             58431 non-null  int32         
 2   recommend           58431 non-null  bool          
 3   review              58431 non-null  object        
 4   user_id             58431 non-null  object        
 5   Año_review          58431 non-null  datetime64[ns]
 6   review_sin_emojis   58431 non-null  object        
 7   sentiment_analysis  58431 non-null  int64         
dtypes: bool(1), datetime64[ns](1), int32(1), int64(1), object(4)
memory usage: 3.0+ MB


In [675]:
dfreviews.rename(columns={'item_id':'Id_juego','recommend':'Recommend','user_id':'Id_usuario'},inplace=True)
dfreviews.columns

Index(['posted', 'Id_juego', 'Recommend', 'review', 'Id_usuario', 'Año_review',
       'review_sin_emojis', 'sentiment_analysis'],
      dtype='object')

In [676]:
dfreviews.drop(columns=['posted','review','review_sin_emojis'],inplace=True)
dfreviews.columns

Index(['Id_juego', 'Recommend', 'Id_usuario', 'Año_review',
       'sentiment_analysis'],
      dtype='object')

In [677]:
dfreviews.to_csv('revies_por_usuario.csv',index=False)